<img src='https://www.osicild.org/uploads/1/2/2/7/122798879/editor/kaggle-v01-clipped.png?1569346633'>
<h1><center>OSIC Pulmonary Fibrosis Progression - EDA</center><h1>

# 0.注釈
  本記事は https://www.kaggle.com/piantic/osic-pulmonary-fibrosis-progression-basic-eda を日本語に訳したものになります。<br>
  内容の誤りや誤訳については予めご容赦ください。
    
# 1. <a id='Introduction'>導入 🃏 </a>
    
###  1.1 肺線維症とは?
* [肺線維症とは、肺の組織が傷ついて瘢痕化することで起こる肺の病気です。](https://www.mayoclinic.org/diseases-conditions/pulmonary-fibrosis/symptoms-causes/syc-20353690)  <br>この肥厚して硬くなった組織は、あなたの肺が正常に働くことをより困難にします。<br>あなたがこのタイプの肺の病気についてさらに知りたい場合は、有益なビデオを下にリンクしています。

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/AfK9LPNj-Zo" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

###  1.2 OSIC Pulmonary Fibrosis Progression コンペティションとは?
- このコンペティションでは、肺のCTスキャンに基づいて患者の肺機能の低下の重症度を予測します。吸い込んだ空気と吐いた空気の量を測定するスピロメーターの出力に基づいて肺機能を判断します。課題は、画像、メタデータ、ベースラインFVCを入力として、機械学習技術を用いて予測を行うことです。
    
    
###  1.3 我々は何をすべきか?
肺のCTスキャンに基づいて、患者さんの肺機能の低下の重症度を予測します。
つまり、各患者さんの最終的な3回のFVC測定値と、その予測の信頼値を予測します。
    
- 本大会のリーダーボードは、テストデータの約1%->15%で算出しています。他の99%->85%で計算されますので、最終的な順位は異なる可能性があります。
    
###  1.4 メトリック:ラプラス対数尤度
![](https://i.imgur.com/tEIZvli.png)
- 画像クレジット: https://en.wikipedia.org/wiki/Laplace_distribution
    
- このコンペティションの評価指標は、ラプラス対数尤度を修正したものです。
予測値は、ラプラス対数尤度の修正版で評価されます。テストセットの各標本について、`FVC` と `Confidence` の尺度 (標準偏差σ) を予測します。

    70より小さい`Confidence`値は切り取られます。

    $\large \sigma_{clipped} = max(\sigma, 70),$

    また、1000を超えるエラーは、大きなエラーを避けるために切り取られています。

    $\large \Delta = min ( |FVC_{true} - FVC_{predicted}|, 1000 ),$

    メトリックは次のように定義されています。

    $\Large metric = -   \frac{\sqrt{2} \Delta}{\sigma_{clipped}} - \ln ( \sqrt{2} \sigma_{clipped} ).$


それについての詳細は [Evaluation Page](https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/overview/evaluation)にあります。

もしこれが新鮮なもので、あなたにとって価値のあるものだったと感じたら、<font color='orange'>upvoting</font>を検討してみてください。 😄

## <font size='5' color='blue'>目次</font> 



* [探索的データ分析の基礎](#1)  
    * [はじめに - ライブラリのインポート]()
    * [train.csvの読み込み]()
    
 
* [データの探索](#2)   
     * [訓練・テストデータをチェック]()
     * [ユニークな患者(ID)]()
     * ['SmokingStatus' カラムの説明]()
     * [週の分布]()
     * [FVC - 強制的な生命維持能力]()
     * [パーセント列の探索]()
     * [性別の分布]()
     * [患者の重なり]()
     
 
* [画像の可視化 : DECOM](#3)    
     * [一つのDECOM画像と情報を可視化]()
     * [複数のDECOM画像と情報を可視化]()
     * [gifを用いて可視化]()
     
     
* [DIOCOMファイルの抽出 Info.](#4)


* [Pandas プロファイリング](#5)
     * [Train.csvに対するPandasのプロファイリングレポート]()
     * [Test.csvに対するPandasのプロファイリングレポート]()


# 2. <a id='importing'>必要なライブラリのインポート📗</a> 

In [ ]:
import os
from os import listdir
import pandas as pd
import numpy as np
import glob
import tqdm
from typing import Dict
import matplotlib.pyplot as plt
%matplotlib inline

# plotly
!pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# color
from colorama import Fore, Back, Style

import seaborn as sns
sns.set(style="whitegrid")

# pydicom
import pydicom

# 警告の抑制 
import warnings
warnings.filterwarnings('ignore')

# プロットの設定
plt.style.use('fivethirtyeight')
plt.show()

# 3. <a id='reading'>train.csvの読み込み 📚</a>

In [ ]:
# 利用可能なファイル一覧
list(os.listdir("../input/osic-pulmonary-fibrosis-progression"))

In [ ]:
IMAGE_PATH = "../input/osic-pulmonary-fibrosis-progressiont/"

train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

print(Fore.YELLOW + 'Training data shape: ',Style.RESET_ALL,train_df.shape)
train_df.head(5)

In [ ]:
train_df.groupby(['SmokingStatus']).count()['Sex'].to_frame()

# 4. <a id='basic'>基本的なデータの探索 🏕️</a> 

## **一般的な情報**

In [ ]:
# NULL値とデータ型
print(Fore.YELLOW + 'Train Set !!',Style.RESET_ALL)
print(train_df.info())
print('-------------')
print(Fore.BLUE + 'Test Set !!',Style.RESET_ALL)
print(test_df.info())

Percentカラムの型はfloat64です。

### 欠損値

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

train_dfとtest_dfには欠損値はありません。

In [ ]:
# データセットに含まれる患者の総数(train+test)

print(Fore.YELLOW +"Total Patients in Train set: ",Style.RESET_ALL,train_df['Patient'].count())
print(Fore.BLUE +"Total Patients in Test set: ",Style.RESET_ALL,test_df['Patient'].count())

 `5` : テストセットの患者数

## ユニークな患者(ID)

In [ ]:
print(Fore.YELLOW + "患者IDの総数は",Style.RESET_ALL,f"{train_df['Patient'].count()},", Fore.BLUE + "これらの中から一意の ID は", Style.RESET_ALL, f"{train_df['Patient'].value_counts().shape[0]}.")

In [ ]:
train_patient_ids = set(train_df['Patient'].unique())
test_patient_ids = set(test_df['Patient'].unique())

train_patient_ids.intersection(test_patient_ids)

テストデータにはすでに `5` の患者が存在しており、訓練データにも同様に見つけることができる。

In [ ]:
columns = train_df.keys()
columns = list(columns)
print(columns)

### **患者数**

In [ ]:
train_df['Patient'].value_counts().max()

訓練データでは、1つの「Patient」に対して複数の行があります。なぜならPatientは週数、FVC、Percentが異なるためです。

In [ ]:
test_df['Patient'].value_counts().max()

テストデータでは、1つのPatientが表示され、Patient idが一意であることを意味しています。

In [ ]:
np.quantile(train_df['Patient'].value_counts(), 0.75) - np.quantile(test_df['Patient'].value_counts(), 0.25)

In [ ]:
print(np.quantile(train_df['Patient'].value_counts(), 0.95))
print(np.quantile(test_df['Patient'].value_counts(), 0.95))

## **トレーニング画像フォルダの患者数と画像数**
* https://www.kaggle.com/yeayates21/osic-simple-image-eda

In [ ]:
files = folders = 0

path = "/kaggle/input/osic-pulmonary-fibrosis-progression/train"

for _, dirnames, filenames in os.walk(path):
  # ^ これの意味は 「この値は使わない」
    files += len(filenames)
    folders += len(dirnames)
#print(Fore.YELLOW +"Total Patients in Train set: ",Style.RESET_ALL,train_df['Patient'].count())
print(Fore.YELLOW +f'{files:,}',Style.RESET_ALL,"files/images, " + Fore.BLUE + f'{folders:,}',Style.RESET_ALL ,'folders/patients')

In [ ]:
files = []
for _, dirnames, filenames in os.walk(path):
  # ^ これの意味は 「この値は使わない」
    files.append(len(filenames))

print(Fore.YELLOW +f'{round(np.mean(files)):,}',Style.RESET_ALL,'average files/images per patient')
print(Fore.BLUE +f'{round(np.max(files)):,}',Style.RESET_ALL, 'max files/images per patient')
print(Fore.GREEN +f'{round(np.min(files)):,}',Style.RESET_ALL,'min files/images per patient')

# 5. <a id='details'>データ探索の詳細 🎠</a> 

## 個々の患者データフレームの作成

`175` のユニークな患者に対して、新しいデータフレームを作成します。

Thanks [@wjdanalharthi](https://www.kaggle.com/wjdanalharthi)

In [ ]:
patient_df = train_df[['Patient', 'Age', 'Sex', 'SmokingStatus']].drop_duplicates()
patient_df.head()

こちらを利用できます:
- https://www.kaggle.com/redwankarimsony/pulmonary-fibrosis-progression-interactive-eda

In [ ]:
# ユニークな患者リストとそのプロパティを作成する
train_dir = '../input/osic-pulmonary-fibrosis-progression/train/'
test_dir = '../input/osic-pulmonary-fibrosis-progression/test/'

patient_ids = os.listdir(train_dir)
patient_ids = sorted(patient_ids)

#新しい行の作成
no_of_instances = []
age = []
sex = []
smoking_status = []

for patient_id in patient_ids:
    patient_info = train_df[train_df['Patient'] == patient_id].reset_index()
    no_of_instances.append(len(os.listdir(train_dir + patient_id)))
    age.append(patient_info['Age'][0])
    sex.append(patient_info['Sex'][0])
    smoking_status.append(patient_info['SmokingStatus'][0])

#患者情報のデータフレームの作成    
patient_df = pd.DataFrame(list(zip(patient_ids, no_of_instances, age, sex, smoking_status)), 
                                 columns =['Patient', 'no_of_instances', 'Age', 'Sex', 'SmokingStatus'])
print(patient_df.info())
patient_df.head()

## 'SmokingStatus' カラムの探索

In [ ]:
patient_df['SmokingStatus'].value_counts()

In [ ]:
patient_df['SmokingStatus'].value_counts().iplot(kind='bar',
                                              yTitle='Counts', 
                                              linecolor='black', 
                                              opacity=0.7,
                                              color='blue',
                                              theme='pearl',
                                              bargap=0.5,
                                              gridcolor='white',
                                              title='Distribution of the SmokingStatus column in the Unique Patient Set')

`118` : 元喫煙者

`49` : 一度も吸ったことがない

`9` : 喫煙者

## 週の分布

In [ ]:
train_df['Weeks'].value_counts().head()

In [ ]:
train_df['Weeks'].value_counts().iplot(kind='barh',
                                      xTitle='Counts(Weeks)', 
                                      linecolor='black', 
                                      opacity=0.7,
                                      color='#FB8072',
                                      theme='pearl',
                                      bargap=0.2,
                                      gridcolor='white',
                                      title='Distribution of the Weeks in the training set')

In [ ]:
train_df['Weeks'].iplot(kind='hist',
                              xTitle='Weeks', 
                              yTitle='Counts',
                              linecolor='black', 
                              opacity=0.7,
                              color='#FB8072',
                              theme='pearl',
                              bargap=0.2,
                              gridcolor='white',
                              title='Distribution of the Weeks in the training set')

Weeksにはマイナスの値もあります。

なぜなら、WeeksはベースラインCTの前後の相対的な週数だからです。

## 週の年齢分布

In [ ]:
fig = px.scatter(train_df, x="Weeks", y="Age", color='Sex')
fig.show()

## FVC - 強制的な生命維持能力

 強制バイタル容量（FVC）、すなわち吐く空気の量。
 - 記録された肺活量（ml)

In [ ]:
train_df['FVC'].value_counts()

In [ ]:
train_df['FVC'].iplot(kind='hist',
                      xTitle='Lung Capacity(ml)', 
                      linecolor='black', 
                      opacity=0.8,
                      color='#FB8072',
                      bargap=0.5,
                      gridcolor='white',
                      title='Distribution of the FVC in the training set')

### FVCとパーセントの比較

In [ ]:
fig = px.scatter(train_df, x="FVC", y="Percent", color='Age')
fig.show()

FVC は Percent と直線的な関係があるようで、理にかなっています。

### FVCと年齢の比較

In [ ]:
fig = px.scatter(train_df, x="FVC", y="Age", color='Sex')
fig.show()

男性は年齢に関係なく女性よりもFVCが高いです。

### FVCと週の比較

In [ ]:
fig = px.scatter(train_df, x="FVC", y="Weeks", color='SmokingStatus')
fig.show()

### ある患者におけるFVCと週の比較

In [ ]:
patient = train_df[train_df.Patient == 'ID00228637202259965313869']
fig = px.line(patient, x="Weeks", y="FVC", color='SmokingStatus')
fig.show()

タバコを吸ったことがない人は、喫煙者よりもFVCが低い。元喫煙者の中には非常に高いFVCを持つ人もいます。

## パーセント

患者のFVCを、類似した特徴を持つ人の典型的なFVCのパーセンテージとして近似した計算フィールド。

In [ ]:
train_df['Percent'].value_counts()

In [ ]:
train_df['Percent'].iplot(kind='hist',bins=30,color='blue',xTitle='Percent distribution',yTitle='Count')

### 患者のデータフレームにおけるパーセントと喫煙状態の比較

In [ ]:
df = train_df
fig = px.violin(df, y='Percent', x='SmokingStatus', box=True, color='Sex', points="all",
          hover_data=train_df.columns)
fig.show()

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.violinplot(x = train_df['SmokingStatus'], y = train_df['Percent'], palette = 'Reds')
ax.set_xlabel(xlabel = 'Smoking Habit', fontsize = 15)
ax.set_ylabel(ylabel = 'Percent', fontsize = 15)
ax.set_title(label = 'Distribution of Smoking Status Over Percentage', fontsize = 20)
plt.show()

In [ ]:
fig = px.scatter(train_df, x="Age", y="Percent", color='SmokingStatus')
fig.show()

### ある患者におけるFVCと週の比較

In [ ]:
patient = train_df[train_df.Patient == 'ID00228637202259965313869']
fig = px.line(patient, x="Weeks", y="Percent", color='SmokingStatus')
fig.show()

## ユニークな患者の年齢分布

In [ ]:
patient_df['Age'].iplot(kind='hist',bins=30,color='red',xTitle='Ages of distribution',yTitle='Count')

### 患者のデータフレームにおける年齢と喫煙状態の比較

In [ ]:
patient_df['SmokingStatus'].value_counts()

In [ ]:
plt.figure(figsize=(16, 6))
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Ex-smoker', 'Age'], label = 'Ex-smoker',shade=True)
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Never smoked', 'Age'], label = 'Never smoked',shade=True)
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Currently smokes', 'Age'], label = 'Currently smokes', shade=True)

# プロットのラベル付け
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.violinplot(x = patient_df['SmokingStatus'], y = patient_df['Age'], palette = 'Reds')
ax.set_xlabel(xlabel = 'Smoking habit', fontsize = 15)
ax.set_ylabel(ylabel = 'Age', fontsize = 15)
ax.set_title(label = 'Distribution of Smokers over Age', fontsize = 20)
plt.show()

### 患者のデータフレームにおける年齢と性別の比較

In [ ]:
plt.figure(figsize=(16, 6))
sns.kdeplot(patient_df.loc[patient_df['Sex'] == 'Male', 'Age'], label = 'Male',shade=True)
sns.kdeplot(patient_df.loc[patient_df['Sex'] == 'Female', 'Age'], label = 'Female',shade=True)
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

## 性別の分布

In [ ]:
patient_df['Sex'].value_counts()

In [ ]:
patient_df['Sex'].value_counts().iplot(kind='bar',
                                          yTitle='Count', 
                                          linecolor='black', 
                                          opacity=0.7,
                                          color='blue',
                                          theme='pearl',
                                          bargap=0.8,
                                          gridcolor='white',
                                          title='Distribution of the Sex column in Patient Dataframe')

`139` : 男性

`37` : 女性

### 患者のデータフレームにおける性別と喫煙状態の比較

In [ ]:
plt.figure(figsize=(16, 6))
a = sns.countplot(data=patient_df, x='SmokingStatus', hue='Sex')

for p in a.patches:
    a.annotate(format(p.get_height(), ','), 
           (p.get_x() + p.get_width() / 2., 
            p.get_height()), ha = 'center', va = 'center', 
           xytext = (0, 4), textcoords = 'offset points')

plt.title('Gender split by SmokingStatus', fontsize=16)
sns.despine(left=True, bottom=True);

In [ ]:
fig = px.box(patient_df, x="Sex", y="Age", points="all")
fig.show()

## 患者の重なり

In [ ]:
# トレーニングセットの患者IDを抽出
ids_train = train_df.Patient.values
# バリデーションセットの患者IDを抽出
ids_test = test_df.Patient.values
#print(Fore.YELLOW +"Total Patients in Train set: ",Style.RESET_ALL,train_df['Patient'].count())
# ユニークなIDを識別するために、学習セットIDの「セット」データ構造を作成
ids_train_set = set(ids_train)
print(Fore.YELLOW + "There are",Style.RESET_ALL,f'{len(ids_train_set)}', Fore.BLUE + 'unique Patient IDs',Style.RESET_ALL,'in the training set')
# ユニークなIDを識別するために、検証セットIDの「セット」データ構造を作成
ids_test_set = set(ids_test)
print(Fore.YELLOW + "There are", Style.RESET_ALL, f'{len(ids_test_set)}', Fore.BLUE + 'unique Patient IDs',Style.RESET_ALL,'in the test set')

# セット間の交点を見て、患者の重複を識別
patient_overlap = list(ids_train_set.intersection(ids_test_set))
n_overlap = len(patient_overlap)
print(Fore.YELLOW + "There are", Style.RESET_ALL, f'{n_overlap}', Fore.BLUE + 'Patient IDs',Style.RESET_ALL, 'in both the training and test sets')
print('')
print(Fore.CYAN + 'These patients are in both the training and test datasets:', Style.RESET_ALL)
print(f'{patient_overlap}')

`5`の患者はトレーニングデータセットとテストデータセットの両方に存在する。

## train.csvのヒートマップ

In [ ]:
corrmat = train_df.corr() 
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmat, ax = ax, cmap = 'RdYlBu_r', linewidths = 0.5) 

以前の可視化情報と比較してみてください。また、以下のPandas Profilingと比較することがあります。

# 6. <a id='visual'>画像の可視化 : DECOM 🗺️</a>  

`1` の情報を含む画像の種類:

- `.dcm` ファイル: [DICOM files](https://en.wikipedia.org/wiki/DICOM). 医療におけるデジタル画像通信形式で保存されています。超音波やMRIなどのメディカルスキャンの画像＋患者の情報が入っています。

In [ ]:
print(Fore.YELLOW + 'Train .dcm number of images:',Style.RESET_ALL, len(list(os.listdir('../input/osic-pulmonary-fibrosis-progression/train'))), '\n' +
      Fore.BLUE + 'Test .dcm number of images:',Style.RESET_ALL, len(list(os.listdir('../input/osic-pulmonary-fibrosis-progression/test'))), '\n' +
      '--------------------------------', '\n' +
      'There is the same number of images as in train/ test .csv datasets')

DICOMの画像を見てみましょう。

In [ ]:
def plot_pixel_array(dataset, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.grid(False)
    plt.imshow(dataset.pixel_array, cmap='gray') # cmap=plt.cm.bone)
    plt.show()

In [ ]:
# https://www.kaggle.com/schlerp/getting-to-know-dicom-and-the-data
def show_dcm_info(dataset):
    print(Fore.YELLOW + "Filename.........:",Style.RESET_ALL,file_path)
    print()

    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print(Fore.BLUE + "Patient's name......:",Style.RESET_ALL, display_name)
    print(Fore.BLUE + "Patient id..........:",Style.RESET_ALL, dataset.PatientID)
    print(Fore.BLUE + "Patient's Sex.......:",Style.RESET_ALL, dataset.PatientSex)
    print(Fore.YELLOW + "Modality............:",Style.RESET_ALL, dataset.Modality)
    print(Fore.GREEN + "Body Part Examined..:",Style.RESET_ALL, dataset.BodyPartExamined)
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print(Fore.BLUE + "Image size.......:",Style.RESET_ALL," {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print(Fore.YELLOW + "Pixel spacing....:",Style.RESET_ALL,dataset.PixelSpacing)
            dataset.PixelSpacing = [1, 1]
        plt.figure(figsize=(10, 10))
        plt.imshow(dataset.pixel_array, cmap='gray')
        plt.show()
for file_path in glob.glob('../input/osic-pulmonary-fibrosis-progression/train/*/*.dcm'):
    dataset = pydicom.dcmread(file_path)
    show_dcm_info(dataset)
    break # すべてを見るには、これをコメントアウトしてください。

In [ ]:
# https://www.kaggle.com/yeayates21/osic-simple-image-eda

imdir = "/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00123637202217151272140"
print("total images for patient ID00123637202217151272140: ", len(os.listdir(imdir)))

# 最初の画像を順に表示
fig=plt.figure(figsize=(12, 12))
columns = 4
rows = 5
imglist = os.listdir(imdir)
for i in range(1, columns*rows +1):
    filename = imdir + "/" + str(i) + ".dcm"
    ds = pydicom.dcmread(filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(ds.pixel_array, cmap='gray')
plt.show()

In [ ]:
# https://www.kaggle.com/yeayates21/osic-simple-image-eda

imdir = "/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00123637202217151272140"
print("total images for patient ID00123637202217151272140: ", len(os.listdir(imdir)))

# 最初の画像を順に表示
fig=plt.figure(figsize=(12, 12))
columns = 4
rows = 5
imglist = os.listdir(imdir)
for i in range(1, columns*rows +1):
    filename = imdir + "/" + str(i) + ".dcm"
    ds = pydicom.dcmread(filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(ds.pixel_array, cmap='jet')
plt.show()

# gifを使った可視化
* https://www.kaggle.com/danpresil1/dicom-basic-preprocessing-and-visualization

In [ ]:
apply_resample = False

def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # int16に変換 
    # 常に低い値でなければならない (<32k)
    image = image.astype(np.int16)

    # スキャン外ピクセルを 0 に設定
    # 切片は通常-1024
    image[image == -2000] = 0
    
    # ハウンズフィールドのユニット（HU）に変換
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
def set_lungwin(img, hu=[-1200., 600.]):
    lungwin = np.array(hu)
    newimg = (img-lungwin[0]) / (lungwin[1]-lungwin[0])
    newimg[newimg < 0] = 0
    newimg[newimg > 1] = 1
    newimg = (newimg * 255).astype('uint8')
    return newimg

In [ ]:
scans = load_scan('../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/')
scan_array = set_lungwin(get_pixels_hu(scans))

In [ ]:
# 1mmへの再サンプル（任意のステップで、z軸のスライス厚が大きいため、本大会には関係ないかもしれません）

from scipy.ndimage.interpolation import zoom

def resample(imgs, spacing, new_spacing):
    new_shape = np.round(imgs.shape * spacing / new_spacing)
    true_spacing = spacing * imgs.shape / new_shape
    resize_factor = new_shape / imgs.shape
    imgs = zoom(imgs, resize_factor, mode='nearest')
    return imgs, true_spacing, new_shape

spacing_z = (scans[-1].ImagePositionPatient[2] - scans[0].ImagePositionPatient[2]) / len(scans)

if apply_resample:
    scan_array_resample = resample(scan_array, np.array(np.array([spacing_z, *scans[0].PixelSpacing])), np.array([1.,1.,1.]))[0]

In [ ]:
import imageio
from IPython.display import Image

imageio.mimsave("/tmp/gif.gif", scan_array, duration=0.0001)
Image(filename="/tmp/gif.gif", format='png')

# アニメーションを使った可視化
* https://www.kaggle.com/pranavkasela/animating-the-lung-ct-scan

アニメーションには、「gifを使った可視化」で用いたscan_arrayを使用しています。

In [ ]:
import matplotlib.animation as animation

fig = plt.figure()

ims = []
for image in scan_array:
    im = plt.imshow(image, animated=True, cmap="Greys")
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)


In [ ]:
HTML(ani.to_jshtml())

In [ ]:
HTML(ani.to_html5_video())

# 7. <a id='extract'>データフレーム内のDIOCOMファイル情報の抽出 🌊</a>

* https://www.kaggle.com/trsekhar123/nb-to-extract-metadata-and-resize-images-train

In [ ]:
def extract_dicom_meta_data(filename: str) -> Dict:
    # イメージの読み込み
    
    image_data = pydicom.read_file(filename)
    img=np.array(image_data.pixel_array).flatten()
    row = {
        'Patient': image_data.PatientID,
        'body_part_examined': image_data.BodyPartExamined,
        'image_position_patient': image_data.ImagePositionPatient,
        'image_orientation_patient': image_data.ImageOrientationPatient,
        'photometric_interpretation': image_data.PhotometricInterpretation,
        'rows': image_data.Rows,
        'columns': image_data.Columns,
        'pixel_spacing': image_data.PixelSpacing,
        'window_center': image_data.WindowCenter,
        'window_width': image_data.WindowWidth,
        'modality': image_data.Modality,
        'StudyInstanceUID': image_data.StudyInstanceUID,
        'SeriesInstanceUID': image_data.StudyInstanceUID,
        'StudyID': image_data.StudyInstanceUID, 
        'SamplesPerPixel': image_data.SamplesPerPixel,
        'BitsAllocated': image_data.BitsAllocated,
        'BitsStored': image_data.BitsStored,
        'HighBit': image_data.HighBit,
        'PixelRepresentation': image_data.PixelRepresentation,
        'RescaleIntercept': image_data.RescaleIntercept,
        'RescaleSlope': image_data.RescaleSlope,
        'img_min': np.min(img),
        'img_max': np.max(img),
        'img_mean': np.mean(img),
        'img_std': np.std(img)}

    return row

In [ ]:
train_image_path = '/kaggle/input/osic-pulmonary-fibrosis-progression/train'
train_image_files = glob.glob(os.path.join(train_image_path, '*', '*.dcm'))

meta_data_df = []
for filename in tqdm.tqdm(train_image_files):
    try:
        meta_data_df.append(extract_dicom_meta_data(filename))
    except Exception as e:
        continue

dcomファイルのmeta要素はファイルごとに微妙に違うようです。そのためExceptionを使っています。
- 気をつけてください！

In [ ]:
# dict から pd.DataFrame への変換
meta_data_df = pd.DataFrame.from_dict(meta_data_df)
meta_data_df.head()

下のコードはまだ意味があるので、そのままにしておきます。

In [ ]:
# ソース: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/154658
folder='train'
PATH='../input/osic-pulmonary-fibrosis-progression/'

last_index = 2

column_names = ['image_name', 'dcm_ImageOrientationPatient', 
                'dcm_ImagePositionPatient', 'dcm_PatientID',
                'dcm_PatientName', 'dcm_PatientSex'
                'dcm_rows', 'dcm_columns']

def extract_DICOM_attributes(folder):
    patients_folder = list(os.listdir(os.path.join(PATH, folder)))
    df = pd.DataFrame()
    
    i = 0
    
    for patient_id in patients_folder:
   
        img_path = os.path.join(PATH, folder, patient_id)
        
        print(img_path)
        
        images = list(os.listdir(img_path))
        
        #df = pd.DataFrame()

        for image in images:
            image_name = image.split(".")[0]

            dicom_file_path = os.path.join(img_path,image)
            dicom_file_dataset = pydicom.read_file(dicom_file_path)
                
            '''
            print(dicom_file_dataset.dir("pat"))
            print(dicom_file_dataset.data_element("ImageOrientationPatient"))
            print(dicom_file_dataset.data_element("ImagePositionPatient"))
            print(dicom_file_dataset.data_element("PatientID"))
            print(dicom_file_dataset.data_element("PatientName"))
            print(dicom_file_dataset.data_element("PatientSex"))
            '''
            
            imageOrientationPatient = dicom_file_dataset.ImageOrientationPatient
            #imagePositionPatient = dicom_file_dataset.ImagePositionPatient
            patientID = dicom_file_dataset.PatientID
            patientName = dicom_file_dataset.PatientName
            patientSex = dicom_file_dataset.PatientSex
        
            rows = dicom_file_dataset.Rows
            cols = dicom_file_dataset.Columns
            
            #print(rows)
            #print(columns)
            
            temp_dict = {'image_name': image_name, 
                                    'dcm_ImageOrientationPatient': imageOrientationPatient,
                                    #'dcm_ImagePositionPatient':imagePositionPatient,
                                    'dcm_PatientID': patientID, 
                                    'dcm_PatientName': patientName,
                                    'dcm_PatientSex': patientSex,
                                    'dcm_rows': rows,
                                    'dcm_columns': cols}


            df = df.append([temp_dict])
            
        i += 1
        
        if i == last_index:
            break
            
    return df


In [ ]:
extract_DICOM_attributes('train')

dcomファイルのmeta要素はファイルごとに微妙に違うようです。
- 気をつけてください!

# <a id='etc'>補論 - Pandas プロファイリング 🌤️</a>

In [ ]:
import pandas_profiling as pdp

In [ ]:
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
profile_train_df = pdp.ProfileReport(train_df)

In [ ]:
profile_train_df

In [ ]:
profile_test_df = pdp.ProfileReport(test_df)

In [ ]:
profile_test_df

## このカーネルが役に立ったら、<font color='orange'>please upvote</font>!
- また次回お会いしましょう!

# 参考
- https://www.kaggle.com/tarunpaparaju/siim-isic-melanoma-eda-pytorch-baseline
- https://www.kaggle.com/andradaolteanu/siim-melanoma-competition-eda-augmentations
- https://www.kaggle.com/parulpandey/melanoma-classification-eda-starter
- https://www.kaggle.com/yeayates21/osic-simple-image-eda
- https://www.kaggle.com/aviralpamecha/osic-in-depth-and-advanced-analysis
- https://www.kaggle.com/danpresil1/dicom-basic-preprocessing-and-visualization